In [13]:
raw = open("data/2.txt", "rb").read().decode()
[i for i in raw.split('\r\n') if i.replace(" ", "")]

['\t\t\t\t就业率',
 '年份    本科毕业生人数\t非师范专业\t就业率\t 非师范生就业率',
 '\t',
 '2015\t  2127 \t\t    1707\t\t98.87%\t        98.59%',
 '2016\t  2156\t\t    1701\t\t98.89%\t        98.59%',
 '2017\t  2455\t\t    2017\t\t99.31%\t        98.46%',
 '2018\t  2486\t\t    1701\t\t98.11%\t        97.71%',
 '2019\t  2486\t\t    1701\t\t98.39%\t        98.04%',
 '\t\t\t      非师范生毕业去向（人数）',
 '年份\t    签约\t\t国内升学\t\t出国（境）\t灵活就业\t\t待就业',
 '2015\t    261\t\t   707\t\t   265\t\t    450\t\t   24',
 '2016\t    235\t\t   737\t\t   296\t\t    409\t\t   24',
 '2017\t    223\t\t   872\t\t   386\t\t    519\t\t   17',
 '2018\t    235\t\t   737\t\t   296\t\t    409\t\t   24',
 '2019\t    169\t\t   889\t\t   403\t\t    541\t\t   40',
 '\t\t\t      非师范生毕业去向（百分数）',
 '年份\t    签约\t\t国内升学\t\t出国（境）\t灵活就业\t\t待就业',
 '2015\t    15.29%\t\t41.42%\t\t15.52%\t\t26.36%\t\t1.41%',
 '2016\t    13.82%\t\t43.33%\t\t17.40%\t\t24.04%\t\t1.42%',
 '2017\t    11.06%\t\t43.23%\t\t19.14%\t\t25.73%\t\t 0.84%',
 '2018\t    13.82%\t\t43.33%\t\t17.40%\t\